# csv파일 읽고 쓰기

## 동일 디렉토리의 csv파일 중 가장 최신 파일을 읽기

In [ ]:
import csv
import glob
import os

list_of_files = glob.glob('/Users/henry/Downloads/*.csv')
latest_file = max(list_of_files, key=os.path.getctime)

with open(latest_file, newline='', encoding='cp949') as csvfile:
    spamreader = csv.reader(csvfile, delimiter=' ', quotechar='|')
    for row in spamreader:
        print(', '.join(row))

## pandas로 csv파일 읽기

In [ ]:
import pandas as pd
import glob
import os

list_of_files = glob.glob('/Users/henry/Downloads/*.csv')
latest_file = max(list_of_files, key=os.path.getctime)
df = pd.read_csv(latest_file, encoding="cp949")

print(df)

## pandas로 csv 파일 읽고 json으로 변환하기

In [ ]:
import pandas as pd
import glob
import os
import json

list_of_files = glob.glob('/Users/henry/Downloads/*.csv')
latest_file = max(list_of_files, key=os.path.getctime)
df = pd.read_csv(latest_file, encoding="cp949")
df = df.loc[:,['종목코드','종목명']]
df = df.set_index('종목코드')
df.columns = ['name']
print(df.shape[0])
result = df.to_dict("index")

with open('stockcode.json', 'w') as outfile:
    json.dump(result, outfile, ensure_ascii=False)

In [94]:
try:
    df2 = pd.read_json("stockcode.json", orient='index')
    deletedCode = df.index.difference(df2.index)
    addedCode = df2.index.difference(df.index)

    print('--- 삭제 목록 ---')
    print('\n'.join(map(str, deletedCode)))
    print('--- 추가 목록 ---')
    print('\n'.join(map(str, addedCode)))

except ValueError:
    print("not find stockcode")


--- 삭제 목록 ---

--- 추가 목록 ---
000021
000023


In [ ]:
import json
import time

with open("stockcode.json") as code:
    data = json.load(code)

for code in data:
    print(code)
    time.sleep(3)

## 서울특별시 부동산 실거래가 정보
http://data.seoul.go.kr/dataList/OA-15548/S/1/datasetView.do;jsessionid=BECD8117AC7AAD4DFB8F7878AE400FA2.new_portal-svr-11

In [2]:
import pandas as pd

df = pd.read_csv("서울특별시_부동산_실거래가_정보_2020년.csv", encoding="cp949")
df.head()

/Users/henry/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (1) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,실거래가아이디,지번코드,시군구코드,자치구명,법정동코드,법정동명,신고년도,업무구분코드,업무구분,물건번호,대지권면적,건물면적,관리구분코드,층정보,건물주용도코드,건물주용도,물건금액,건축년도,건물명
0,11290-2020-4-0000066-1,1129013800101440024,11290,성북구,1129013800,장위동,2020,4,신고,1,0.0,59.92,2,5.0,2001,아파트,749000000,0.0,래미안 장위포레카운티
1,11290-2020-4-0000628-1,1129013400105080016,11290,성북구,1129013400,길음동,2020,4,신고,1,0.0,84.77,2,18.0,2001,아파트,1200000000,0.0,롯데캐슬 클라시아
2,11530-2020-4-0014284-1,1153010200107400029,11530,구로구,1153010200,구로동,2020,4,신고,1,NaN,36.90,0,1.0,2001,아파트,105000000,1994.0,궁전아트빌라
3,11170-2020-4-0005040-1,1117011500101930000,11170,용산구,1117011500,산천동,2020,4,신고,1,NaN,59.55,0,2.0,2001,아파트,830000000,2001.0,리버힐삼성
4,11170-2020-4-0001553-1,1117012900101930003,11170,용산구,1117012900,이촌동,2020,4,신고,1,NaN,64.43,0,5.0,2001,아파트,735000000,1971.0,강변


## 법정동코드 읽기

https://www.code.go.kr/stdcode/regCodeL.do

In [ ]:
import csv
import glob
import os

with open('법정동코드 전체자료.txt', newline='', encoding='cp949') as csvfile:
    spamreader = csv.reader(csvfile, delimiter=' ', quotechar='\t')
    for row in spamreader:
        print(', '.join(row))

## 법정동 코드 읽고 json으로 변환하기

In [126]:
import pandas as pd
import json


def define_district(full_dong):
    dong = full_dong.split(" ")
    return len(dong)

df = pd.read_csv('법정동코드 전체자료.txt', encoding="cp949", delimiter='\t')
df['법정동코드'] = df['법정동코드'].astype(str).str[:5]

df = df[df['폐지여부']=='존재']
df = df.drop(['폐지여부'], axis=1)

df['법정동명'] = df['법정동명'].apply(lambda x: x if len(x.split(" ")) > 1 else "" )
df = df[df['법정동명'] != '']
df = df.drop_duplicates('법정동코드')
df = df.set_index('법정동코드')
result = df.to_dict()
data = result.get('법정동명')
with open("district_code.json", "w") as w:
    json.dump(data, w, ensure_ascii=False)

In [130]:
import json
with open("district_code.json", "r") as r:
    data = json.load(r)

data.get('11110')

'서울특별시 종로구'